In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv



<img src="https://i.ibb.co/1T5Wbn6/Banner-para-Linkedin-capa-de-perfil-para-programador.png"  alt="KCode Banner"  />

<a href="https://www.linkedin.com/in/kauebr/"><img src="https://cdn-icons-png.flaticon.com/512/174/174857.png" width="30" ></a> <a href="mailto: kauemandarino@gmail.com"><img src="https://cdn-icons-png.flaticon.com/512/5968/5968534.png" width="30" ></a><a href="
https://api.whatsapp.com/send?phone=5541991835800&text=Oi"> <img src="https://imagepng.org/wp-content/uploads/2017/08/whatsapp-icone-1.png" width="30" ></a><a href="https://github.com/kauebr/"><img src="https://github.githubassets.com/images/modules/logos_page/Octocat.png" width="30" ></a>
Processament

# Competição Kaggle, Processamento de Linguagem Natural - NLP

## Tweets de desastres naturais
* Na competição temos um conjunto de 10.000   
* Os tweets foram classificados manualmente como:
   - um tweet normal(0), 'Oh que dia lindo!'
   - um tweet relatando um desastre natural(1), 'Incêndio na BR 277'  
   (os tweets são em inglês)

### Importando as bibliotecas necessárias:

In [2]:
import numpy as np 
import pandas as pd 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

### Criando nossos conjuntos de treino e teste:

In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

### Olhada nos dados
* Uma vez carregados os dados vou dar uma olhada nos tweets

In [4]:
# Exibindo 10 tweets normais(0)
train_df.loc[train_df['target'] == 0].head(10)


,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0
17,25,NaN,NaN,Summer is lovely,0
18,26,NaN,NaN,My car is so fast,0
19,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0
20,31,NaN,NaN,this is ridiculous....,0
21,32,NaN,NaN,London is cool ;),0
22,33,NaN,NaN,Love skiing,0
23,34,NaN,NaN,What a wonderful day!,0
24,36,NaN,NaN,LOOOOOOL,0


In [5]:
# Exibindo 10 tweets desastre(1)
train_df.loc[train_df['target'] == 1].head(10)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


### Quero dar uma olhada no tamanho do meu conjunto de teste

In [6]:
train_df.shape

(7613, 5)

>Interessante aqui que temos, 7.6 mil tweets para analisar, é um volume grande, deve ter dado uma trabalheira classificar manualmente, mas devido a isto, tendemos a obter um grau grande de precisão, jogando lá embaixo nosso erro amostral.

#### Construindo vetores
A teoria por trás do modelo que iremos construir neste notebook é bastante simples: as palavras contidas em cada tweet são um bom indicador se eles se referem ou não a um desastre real (isso não é totalmente correto, mas é um ótimo ponto de partida).

Vamos usar o CountVectorizer do scikit-learn para contar as palavras em cada tweet e transformá-las em dados que nosso modelo de aprendizado de máquina pode processar.

Observação: um vetor, nesse contexto, é um conjunto de números com os quais um modelo de aprendizado de máquina pode trabalhar. Vamos analisar um vetor em apenas um segundo.

In [7]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## Vamos obter a contagem das primeiras 5 postagens no conjunto de dados.
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
## Nós usamos .todense() aqui porque esses vetores são "esparços" (apenas os elementos não nulos são mantidos para economizar espaço).
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


Podemos notar acima que existem 54 palavras únicas (ou "tokens") nos primeiros cinco tweets.
O primeiro tweet contém apenas alguns desses tokens únicos - todas as contagens não nulas acima são os tokens que EXISTEM no primeiro tweet.


Existem 54 palavras únicas (ou "tokens") nos primeiros cinco tweets.
O primeiro tweet contém apenas alguns desses tokens únicos - todas as contagens não nulas acima são os tokens que EXISTEM no primeiro tweet.
Agora vamos criar vetores para todos os nossos tweets.

### Agora vamos criar vetores para todos os nossos tweets.

In [9]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

# Observe que NÃO estamos usando .fit_transform() aqui. 
# Ao usar apenas .transform(), garantimos que os tokens nos vetores de treinamento
# sejam os únicos mapeados para os vetores de teste - 
# ou seja, que os vetores de treinamento e teste usem o mesmo conjunto de tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

Nosso modelo
Como mencionei anteriormente, acredito que as palavras contidas em cada tweet são um bom indicador se eles se referem ou não a um desastre real. A presença de uma palavra específica (ou conjunto de palavras) em um tweet pode estar diretamente relacionada a se o tweet desastre ou não.

Agora vamos construir o modelo!

In [10]:
## Nossos vetores são realmente grandes, então queremos ajustar os pesos do nosso modelo.
## Em direção a 0, sem descartar completamente palavras diferentes - ridge regression 
clf = linear_model.RidgeClassifier()

Vamos testar nosso modelo e ver como ele se sai nos dados de treinamento. Para isso, usaremos a validação cruzada - em que treinamos uma parte dos dados conhecidos e, em seguida, validamos com o restante. Se fizermos isso várias vezes (com diferentes partes), podemos ter uma boa ideia de como um determinado modelo ou método se comporta.

A métrica para esta competição é o F1, então vamos usá-lo aqui.

In [11]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.5642787 , 0.64082434])

> As pontuações acima estão baixas, mas aceitavéis, para melhorar isto existem vários metódos a serem aplicados em notebooks posteriorres. Agora vou fazer as predições enviar.

In [12]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [13]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [14]:
sample_submission["target"] = clf.predict(test_vectors)

In [15]:
# Exibindo amostra das predições
sample_submission.head(10)

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [16]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [17]:
# Registrando as predições na competição
sample_submission.to_csv("submission.csv", index=False)

## Tudo pronto
* Porém agora eu vou exportar as predições que eu quero dar uma olhada

In [18]:
predições = test_df.merge(sample_submission[['id', 'target']], on='id', how='right')
predições['target'] = predições['target'].replace({0: 'Tweet normal', 1: 'Tweet desastre'})
predições = predições.drop(['keyword', 'location'], axis=1)
predições.head(3)



,id,text,target
0,0,Just happened a terrible car crash,Tweet normal
1,2,"Heard about #earthquake is different cities, s...",Tweet desastre
2,3,"there is a forest fire at spot pond, geese are...",Tweet desastre


In [19]:
predições.to_csv('predicoes.csv', index=False)


> Tudo pronto, agora só enviar e esperar o resultado